**SAP Project**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
!pip install "torch==2.1.2" tensorboard
!pip install --upgrade \
"transformers==4.38.2" \
"datasets==2.16.1" \
"accelerate==0.26.1" \
"evaluate==0.4.1" \
"bitsandbytes==0.42.0" \
"trl==0.7.11" \
"peft==0.8.2" \
"langchain" \
"sentence-transformers" \
"faiss-cpu"
!pip install unstructured
!pip install pdfminer
!pip install pdfminer.six

  Using cached torch-2.1.2-cp311-cp311-manylinux1_x86_64.whl.metadata (25 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.18.1-py3-none-manylinux1_x86_64.whl.m

In [ ]:
import os
from transformers import pipeline
os.environ['HF_TOKEN'] = 'YOUR_HUGGING_FACE_TOKEN'
hf_token = os.getenv('HF_TOKEN')


In [3]:
!pip install langchain_community
import torch
!pip install langchain
from IPython.display import display_markdown
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from langchain.text_splitter import RecursiveCharacterTextSplitter
from transformers import pipeline
import transformers
from langchain.document_loaders import UnstructuredPDFLoader, PDFMinerLoader, TextLoader
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

In [26]:
model_id = "stabilityai/stablelm-zephyr-3b"
pipeline = transformers.pipeline(
    "text-generation",
    model=model_id, token=hf_token,
    model_kwargs={
        "torch_dtype": torch.float16,
        "quantization_config": {"load_in_4bit": True},
        "low_cpu_mem_usage": True,
    },
)

terminators = [
    pipeline.tokenizer.eos_token_id,
    pipeline.tokenizer.convert_tokens_to_ids("")
]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [27]:
pdf_file_path = "/content/output ds.pdf"

In [28]:
class stablelm_zephyr_3b:
    def __init__(self, pipeline):
        self.pipeline = pipeline

    @staticmethod
    def generate_prompt(query, retrieved_text):
        messages = [
            {"role": "system", "content": "Answer the Question for the Given below context and information and not prior knowledge, only give the output result\n\ncontext:\n\n{}".format(retrieved_text)},
            {"role": "user", "content": query},
        ]
        return pipeline.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

    def generate(self, query, retrieved_context):
        prompt = self.generate_prompt(query, retrieved_context)
        output = pipeline(prompt, max_new_tokens=512, eos_token_id=terminators, do_sample=True, temperature=0.7, top_p=0.9,)
        return output[0]["generated_text"][len(prompt):]

In [29]:
class Langchain_RAG:
    def __init__(self, pdf_file_path):
        self.embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-small-en-v1.5")
        self.pdf_file_path = pdf_file_path
        print("loading pdf file , this may take time to process")
        self.loader = PDFMinerLoader(self.pdf_file_path)
        self.data = self.loader.load()
        print("<< pdf file loaded")
        print("<< chunking")
        text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0, separators=[" ", ",", "\n"])
        self.texts = text_splitter.split_documents(self.data)
        print("<< chunked")
        self.get_vec_value = FAISS.from_documents(self.texts, self.embeddings)
        print("<< saved")
        self.retriever = self.get_vec_value.as_retriever(search_kwargs={"k": 4})
    def __call__(self, query):
        rev = self.retriever.get_relevant_documents(query)
        return "".join([i.page_content for i in rev])

In [30]:
text_gen = stablelm_zephyr_3b(pipeline=pipeline)
retriever = Langchain_RAG(pdf_file_path=pdf_file_path)

loading pdf file , this may take time to process
<< pdf file loaded
<< chunking
<< chunked
<< saved


In [31]:
def ask(query):
    retriever_context = retriever(query)
    result = text_gen.generate(query, retriever_context)
    return result

**Queries : **

In [32]:
display_markdown(ask("give me the best rating shoes with a price under 2000"), raw=True)

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Based on the given context, the MACTREE Men's Genuine Leather Formal Slip On Shoes has the best rating among the listed shoes with a rating of 3.6, and it is sold at a price of Rs.732.46 by Relaxofootwear Ltd. However, this shoe is not under the price of 2000.

To find a shoe with a rating higher than the MACTREE Men's Genuine Leather Formal Slip On Shoes and a price under 2000, we will need to look for shoes with higher ratings that fit the given price criteria. The next best option would be the crocs Unisex-Adult Classic Clogs, which has a rating of 4.7 and costs under 2000, priced at Rs.1493.94 by Cocoblu Retail.

In [33]:
display_markdown(ask("From the Above pdf Who are the Top vendors In the Formal Shoes"), raw=True)

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


To identify the top vendors in the given context, we need to analyze the information provided for each product and find the vendor with the highest rating. Here's a breakdown of the top vendors based on the given information:

1. Arthur Harvey Men's Leather Formal Shoes:
Vendor: Neemans Shoes
Rating: 3.8
Price: Rs.1095.05
Delivery: 2 days
Sizes: 8UK, 9UK, 1UK, 0UK, 1UK, 1UK, 1UK, 2UK
Image Links: [Image Link 1], [Image Link 2]

2. Centrino Mens 8620 Formal:
Vendor A: Cocoblu Retail
Vendor B: Campusactivewear
Ratings: 3.1, 3.5
Prices: Rs.664.06, Rs.661.62
Delivery A: 6 days, 1 days
Sizes: 7UK, 8UK, 9UK, 1UK, 0UK, 1UK, 1UK, 2UK
Image Links: [Image Link 1], [Image Link 2]

3. MACTREE Men Genuine Leather Formal Shoes:
Vendor: Bansal Traders
Rating: 3.0
Price: Rs.754.99
Delivery: 3 days
Sizes: 7UK, 8UK, 9UK, 1UK, 0UK, 1UK, 1UK, 2UK
Image Link: [Image Link 1]

4. Vellinto® Oxford Class Designer Patent Leatherette Shining Lace-Up Party Formal Shoes:
Vendor A: Cocoblu Retail
Vendor B: Radhe Creations
Vendor C: RetailEZ Pvt. Ltd.
Vendor D: Neemans Shoes
Ratings: 4.3, 4.5, 3.9, 3.5
Prices: Rs.599.87, Rs.600.17, Rs.597.0, Rs.595.36
Delivery A: 6 days, 4 days, 4 days, 3 days
Sizes: 7UK, 8UK, 1UK, 0UK, 1UK, 1UK, 1UK, 2UK
Image Links: [Image Link 1], [Image Link 2], [Image Link 3], [Image Link 4]

5. Centrino Mens 8621

In [34]:
!pip install -q gradio

In [18]:
def predict(user_question, history):
  answer = ask(user_question)
  answer = (answer.split("<|assistant|>")[-1]).strip()
  return str(answer)

In [19]:
import gradio as gr
gr.ChatInterface(predict).launch(share=True)

/usr/local/lib/python3.11/dist-packages/gradio/chat_interface.py:339: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://8d4018a5d87cb365d7.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [35]:
import gradio as gr

# Define a wrapper function to use your model pipeline
def predict(question):
    output = pipeline(question, max_new_tokens=256)[0]["generated_text"]
    return output

iface = gr.Interface(fn=predict, inputs="text", outputs="text")
iface.launch(share=True, debug=True)


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://acaf8306a22c4606dc.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
/usr/local/lib/python3.11/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7868 <> https://acaf8306a22c4606dc.gradio.live
